##### Copyright 2018 The TensorFlow Authors.

In [12]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Image classification

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/images/classification"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ja/tutorials/images/classification.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ja/tutorials/images/classification.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ja/tutorials/images/classification.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

Note: これらのドキュメントは私たちTensorFlowコミュニティが翻訳したものです。コミュニティによる 翻訳は**ベストエフォート**であるため、この翻訳が正確であることや[英語の公式ドキュメント](https://www.tensorflow.org/?hl=en)の 最新の状態を反映したものであることを保証することはできません。 この翻訳の品質を向上させるためのご意見をお持ちの方は、GitHubリポジトリ[tensorflow/docs](https://github.com/tensorflow/docs)にプルリクエストをお送りください。 コミュニティによる翻訳やレビューに参加していただける方は、 [docs-ja@tensorflow.org メーリングリスト](https://groups.google.com/a/tensorflow.org/forum/#!forum/docs-ja)にご連絡ください。

このチュートリアルでは、画像から猫または犬を分類する方法を示します。 `tf.keras.Sequential` モデルを使用して画像分類器を構築し、 `tf.keras.preprocessing.image.ImageDataGenerator` を使用してデータをロードします。このチュートリアルでは、以下のコンセプトにしたがって、実践的な経験と感覚を養います。

* `tf.keras.preprocessing.image.ImageDataGenerator` クラスを使用して _データ入力パイプライン_ を構築し、モデルで使用するディスク上のデータを効率的に処理します。
* _過学習（Overfitting）_ —過学習を識別および防止する方法。
* _データ拡張（Data Augmentation）_ および _ドロップアウト（dropout）_ —データパイプラインおよび画像分類モデルに組み込むコンピュータービジョンタスクの過学習と戦うための重要なテクニック。

このチュートリアルは、基本的な機械学習のワークフローに従います。

1. データの調査及び理解
2. 入力パイプラインの構築
3. モデルの構築
4. モデルの学習
5. モデルのテスト
6. モデルの改善とプロセスの繰り返し

元のURL
https://www.tensorflow.org/tutorials/images/classification?hl=ja

2022/2/27
1個目の改良版
pre_normal_r_l.h5の予測モデルが良かったプログラム。

## パッケージのインポート

まずは必要なパッケージをインポートすることから始めましょう。 `os`パッケージはファイルとディレクトリ構造を読み込み、 NumPy は python リストの numpy 配列への変換と必要な行列演算の実行、 `matplotlib.pyplot` はグラフの描画や学習データおよび検証データに含まれる画像の表示、に利用します。

モデルの構築に必要な TensorFlow と Keras クラスをインポートします。

In [13]:
import tensorflow as tf

In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator




import os
import numpy as np
import matplotlib.pyplot as plt

## データの読み込み

データセットのダウンロードから始めます。このチュートリアルでは、 Kaggle の <a href="https://www.kaggle.com/c/dogs-vs-cats/data" target="_blank">Dogs vs Cats</a> データセットをフィルタリングしたバージョンを使用します。データセットのアーカイブバージョンをダウンロードし、"/tmp/"ディレクトリに保存します。

In [15]:
#ドライブ直接読込
PATH = "/content/drive/MyDrive/Colab Notebooks/cats_and_dogs_filtered"




データセットのディレクトリ構造は次のとおりです:

<pre>
<b>cats_and_dogs_filtered</b>
|__ <b>train</b>
    |______ <b>cats</b>: [cat.0.jpg, cat.1.jpg, cat.2.jpg ....]  ＞ gamu（ガムシロップ）
    |______ <b>dogs</b>: [dog.0.jpg, dog.1.jpg, dog.2.jpg ...]  ＞ milk（ミルク）
|__ <b>validation</b>
    |______ <b>cats</b>: [cat.2000.jpg, cat.2001.jpg, cat.2002.jpg ....] ＞ gamu（ガムシロップ）
    |______ <b>dogs</b>: [dog.2000.jpg, dog.2001.jpg, dog.2002.jpg ...] ＞ milk（ミルク）
</pre>

データの内容を抽出した後、学習および検証セットのための適切なファイルパスで変数を設定します。

In [16]:
train_dir = os.path.join(PATH, 'train')
validation_dir = os.path.join(PATH, 'validation')

In [17]:
train_normal_r_dir = os.path.join(train_dir, 'milk')  # 学習用の猫画像のディレクトリ
train_normal_l_dir = os.path.join(train_dir, 'gamu')  # 学習用の犬画像のディレクトリ
validation_normal_r_dir = os.path.join(validation_dir, 'milk')  # 検証用の猫画像のディレクトリ
validation_normal_l_dir = os.path.join(validation_dir, 'gamu')  # 検証用の犬画像のディレクトリ

### データの理解

学習および検証ディレクトリの中にある猫と犬の画像の数を見てみましょう:

In [18]:
num_cats_tr = len(os.listdir(train_normal_r_dir))
num_dogs_tr = len(os.listdir(train_normal_l_dir))

num_cats_val = len(os.listdir(validation_normal_r_dir))
num_dogs_val = len(os.listdir(validation_normal_l_dir))

total_train = num_cats_tr + num_dogs_tr
total_val = num_cats_val + num_dogs_val

FileNotFoundError: ignored

In [19]:
print('total training normal images:', num_cats_tr)
print('total training glaucoma images:', num_dogs_tr)

print('total validation normal images:', num_cats_val)
print('total validation glaucoma images:', num_dogs_val)
print("--")
print("Total training images:", total_train)
print("Total validation images:", total_val)

NameError: ignored

便宜上、データセットの前処理およびネットワークの学習中に使用する変数を設定します。

In [ ]:
batch_size = 16
epochs = 1000
IMG_HEIGHT = 128 #432 #150
IMG_WIDTH = 128 #648 #150

## ***データの準備***

モデルにデータを送る前に、画像を適切に前処理された浮動小数点テンソルにフォーマットします。

1.ディスクから画像を読み取ります。
2.これらの画像のコンテンツをデコードし、RGB値にしたがって適切なグリッド形式に変換します。
3.それらを浮動小数点テンソルに変換します。
4.ニューラルネットワークは小さな入力値を扱う方が適しているため、テンソルを0〜255の値から0〜1の値にリスケーリングします。

幸い、これらすべてのタスクは、 `tf.keras` によって提供される `ImageDataGenerator` クラスで実行できます。この `ImageDataGenerator` はディスクから画像を読み取り、適切なテンソルに前処理を行います。さらに、これらの画像をテンソルのバッチに変換するジェネレータをセットアップします。これは、ネットワーク学習時に便利です。

In [ ]:
train_image_generator = ImageDataGenerator(rescale=1./255) # 学習データのジェネレータ
validation_image_generator = ImageDataGenerator(rescale=1./255) # 検証データのジェネレータ

学習および検証画像のジェネレータを定義したのち、 `flow_from_directory` メソッドはディスクから画像をロードし、リスケーリングを適用し、画像を必要な大きさにリサイズします。

In [ ]:
train_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                           directory=train_dir,
                                                           shuffle=True,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           class_mode='binary')

In [ ]:
val_data_gen = validation_image_generator.flow_from_directory(batch_size=batch_size,
                                                              directory=validation_dir,
                                                              target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                              class_mode='binary')

### 学習用画像の可視化

学習用のジェネレータから画像バッチを抽出して可視化します。（この例では32個の画像を抽出し、そのうち5つを `matplotlib` で描画します。）

In [ ]:
#sample_training_images, _ = next(train_data_gen)

 `next` 関数はデータセットからバッチを返します。 `next` 関数の返り値は `（x_train、y_train）` の形式で、 `x_train` は学習用の特徴量、 `y_train` はそのラベルです。ラベルを破棄して、学習用画像の可視化のみを行います。

In [ ]:
# この関数は、1行5列のグリッド形式で画像をプロットし、画像は各列に配置されます。
#def plotImages(images_arr):
#    fig, axes = plt.subplots(1, 5, figsize=(20,20))
#    axes = axes.flatten()
#    for img, ax in zip( images_arr, axes):
#        ax.imshow(img)
#        ax.axis('off')
#    plt.tight_layout()
#    plt.show()

In [ ]:
#plotImages(sample_training_images[:5])

## モデルの構築

モデルはmax pooling層を伴う3つの畳み込みブロックからなります。さらに `relu` 活性化関数によるアクティベーションを伴う512ユニットの全結合層があります。モデルは、シグモイド活性化関数による2値分類に基づいてクラスに属する確率を出力します。

In [ ]:
#model = Sequential([
#    Conv2D(16, 3, padding='same', activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH ,3)),
#    MaxPooling2D(),
#    Conv2D(32, 3, padding='same', activation='relu'),
#    MaxPooling2D(),
#    Conv2D(64, 3, padding='same', activation='relu'),
#    MaxPooling2D(),
#    Flatten(),
#    Dense(512, activation='relu'),
#    Dense(1, activation='sigmoid')
#])

### モデルのコンパイル
このチュートリアルでは、 *ADAM* オプティマイザーと *binary cross entropy* 損失関数を選択します。各学習エポックの学習と検証の精度を表示するために、`metrics` 引数を渡します。

In [ ]:
#model.compile(optimizer='adam',
#              loss='binary_crossentropy',
#              metrics=['accuracy'])

### モデルの概要

すべてのネットワークのレイヤーを見るには、モデルの `summary` メソッドを利用します:

In [ ]:
#model.summary()

### モデルの学習

`ImageDataGenerator` クラスの `fit_generator` メソッドを使用して、ネットワークを学習します。

In [ ]:
#history = model.fit_generator(
#    train_data_gen,
#    steps_per_epoch=total_train // batch_size,
#    epochs=epochs,
#    validation_data=val_data_gen,
#    validation_steps=total_val // batch_size
#
#)
#
#model.save('./pre_normal_r_l.h5')


### 学習結果の可視化

ネットワークを学習した後、結果を可視化します。

In [ ]:
#acc = history.history['accuracy']
#val_acc = history.history['val_accuracy']
#
#loss = history.history['loss']
#val_loss = history.history['val_loss']
#
#epochs_range = range(epochs)
#
#plt.figure(figsize=(8, 8))
#plt.subplot(1, 2, 1)
#plt.plot(epochs_range, acc, label='Training Accuracy')
#plt.plot(epochs_range, val_acc, label='Validation Accuracy')
#plt.legend(loc='lower right')
#plt.title('Training and Validation Accuracy')
#
#plt.subplot(1, 2, 2)
#plt.plot(epochs_range, loss, label='Training Loss')
#plt.plot(epochs_range, val_loss, label='Validation Loss')
#plt.legend(loc='upper right')
#plt.title('Training and Validation Loss')
#plt.show()

プロットからわかるように、学習セットの精度と検証セットの精度は大幅に外れており、モデルは検証セットで約70％の精度しか達成していません。

何がうまくいかなかったかを見て、モデル全体のパフォーマンスを向上してみましょう。

## 過学習

上記のプロットでは、学習セットの精度は時間とともに直線的に向上していますが、検証セットの精度は学習プロセスの中で約70％あたりで頭打ちになっています。そして、学習と検証の精度の違いが顕著です。これは *過学習* のサインです。

学習サンプルが少ない場合、モデルは学習サンプルに含まれるノイズや不要な詳細から学習してしまい、これによって新しいサンプルに対するモデルの性能に悪影響を与えることがあります。この現象は、過学習として知られています。過学習とは、モデルが新しいデータセットに対して汎化するのが難しい状態をいいます。

学習プロセスにおいて過学習に対抗する手段はいくつかあります。このチュートリアルでは、*データ拡張（data Augmentation）* を使用し、さらにモデルに *ドロップアウト（dropout）* を追加します。

## データ拡張（Data augmentation）

過学習は一般に、学習サンプルが少ない場合に発生します。この問題を解決する方法の1つは、十分な数の学習サンプルが含まれるようにデータセットを拡張することです。データ拡張は、既存の学習サンプルに対してランダムな変換を行い、データセットとして利用できそうな画像を生成するアプローチをとります。このデータ拡張の目的は、学習中にモデルがまったくおなじ画像を2回利用しないようにすることです。これによってモデルをデータのより多くの特徴を利用し、より汎化することができます。

`tf.keras` においては、このデータ拡張を `ImageDataGenerator` クラスを使用して実装します。データセットに対するさまざまな変換を指定することによって、学習プロセス中にそれが適用されます。

### データの拡張と可視化

最初に、ランダムな水平反転による拡張をデータセットに適用し、それぞれの画像が変換後にどのように見えるかを確認します。

### 水平反転の適用

 このデータ拡張を適用するためには、 `ImageDataGenerator` クラスの引数として `horizontal_flip` を渡し、 `True`を設定します。

In [ ]:
#image_gen = ImageDataGenerator(rescale=1./255, horizontal_flip=True)

In [ ]:
#train_data_gen = image_gen.flow_from_directory(batch_size=batch_size,
#                                               directory=train_dir,
#                                               shuffle=True,
#                                               target_size=(IMG_HEIGHT, IMG_WIDTH))

学習サンプルから1つのサンプル画像を取得する作業を5回繰り返して、おなじ画像に5回データ拡張が適用されるようにします。

In [ ]:
#augmented_images = [train_data_gen[0][0][0] for i in range(5)]

In [ ]:
# 上で学習用画像の可視化のために定義、使用されたおなじカスタムプロット関数を再利用する
#plotImages(augmented_images)

### 画像のランダムな回転

回転のデータ拡張を利用して学習用サンプルをランダムに左右45度の範囲で回転させてみましょう。

In [ ]:
image_gen = ImageDataGenerator(rescale=1./255, rotation_range=30) #45

In [ ]:
train_data_gen = image_gen.flow_from_directory(batch_size=batch_size,
                                              directory=train_dir,
                                              shuffle=True,
                                               target_size=(IMG_HEIGHT, IMG_WIDTH))

#augmented_images = [train_data_gen[0][0][0] for i in range(5)]
augmented_images = [train_data_gen[0][0][0] for i in range(20)]

In [ ]:
#plotImages(augmented_images)

### ズームによるデータ拡張の適用

データセットにズームによるデータ拡張を適用して、画像をランダムに最大50％拡大します。

In [ ]:
image_gen = ImageDataGenerator(rescale=1./255, zoom_range=0.15) #0.5

In [ ]:
train_data_gen = image_gen.flow_from_directory(batch_size=batch_size,
                                               directory=train_dir,
                                               shuffle=True,
                                               target_size=(IMG_HEIGHT, IMG_WIDTH))

#augmented_images = [train_data_gen[0][0][0] for i in range(5)]
augmented_images = [train_data_gen[0][0][0] for i in range(20)]

In [ ]:
#plotImages(augmented_images)

### すべてのデータ拡張を同時に利用する

ここまでで紹介したすべてのデータ拡張機能を適用します。ここでは、学習用画像に対して、リスケール、45度の回転、幅シフト、高さシフト、水平反転、ズームを適用しました。

In [ ]:
image_gen_train = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=30,#45,
                    width_shift_range=.15,
                    height_shift_range=.15,
                    #horizontal_flip=True,
                    zoom_range=0.15 #0.5
                    )

In [ ]:
train_data_gen = image_gen_train.flow_from_directory(batch_size=batch_size,
                                                     directory=train_dir,
                                                     shuffle=True,
                                                     target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                     class_mode='binary')

これらのデータ拡張がデータセットにランダムに適用されたときに、一つの画像に対して5回の個別の適用を行った際にそれぞれどのように見えるかを可視化します。

In [ ]:
#augmented_images = [train_data_gen[0][0][0] for i in range(5)]
augmented_images = [train_data_gen[0][0][0] for i in range(20)]
#plotImages(augmented_images)

### 検証データジェネレータの構築

一般に、データ拡張は学習サンプルのみに適用します。今回は、 `ImageDataGenerator` を使用して検証画像に対してリスケールのみを実施し、バッチに変換します。

In [ ]:
image_gen_val = ImageDataGenerator(rescale=1./255)

In [ ]:
val_data_gen = image_gen_val.flow_from_directory(batch_size=batch_size,
                                                 directory=validation_dir,
                                                 target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                 class_mode='binary')

## ドロップアウト（dropout）

過学習を避けるもう一つの方法は、ネットワークに *ドロップアウト* を導入することです。これは、ネットワークにおいて重みを小さくする正則化の方式で、これによって重みの値の分布がより規則的になり、少ない学習データに対する過学習を減らすことができます。ドロップアウトはこのチュートリアルで利用される正則化手法の一つです。

ドロップアウトをレイヤーに適用すると、学習プロセス中に適用されたレイヤーのうちランダムに出力ユニットをドロップアウト（ゼロに設定）します。ドロップアウトは、入力値として0.1、0.2、0.4といった形式の小数をとります。これは、適用されたレイヤーからランダムに出力単位の10％、20％、または40％をドロップアウトすることを意味します。

特定のレイヤーに0.1ドロップアウトを適用すると、各学習エポックにおいて出力ユニットの10％がランダムに0にされます。

この新しいドロップアウト機能を使用したネットワークアーキテクチャを作成し、異なる畳み込みレイヤーや全接続レイヤーに適用してみましょう。

## ドロップアウトを追加した新しいネットワークの構築

ここでは、ドロップアウトを最初と最後の max pool 層に適用します。ドロップアウトを適用すると、各学習エポック中にニューロンの20％がランダムにゼロに設定されます。これにより、学習データセットに対する過学習を避けることができます。

In [ ]:
#model_new = Sequential([
#    Conv2D(512,3, padding='same', activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH ,3)),
#    MaxPooling2D(),
#    Dropout(0.2),#0.2
#    Conv2D(64,3, padding='same', activation='relu'),
#    MaxPooling2D(),
#    Conv2D(32,3, padding='same', activation='relu'),
#    MaxPooling2D(),
#    Dropout(0.2),#0.2
#    Flatten(),
#    Dense(16, activation='relu'),
#    Dense(1, activation='sigmoid')
#])



model_new = Sequential([
    Conv2D(16,3, padding='same', activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH ,3)),
    MaxPooling2D(),
    Dropout(0.2),#0.2
    Conv2D(32,3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64,3, padding='same', activation='relu'),
    MaxPooling2D(),
    Dropout(0.2),#0.2
    Flatten(),
    Dense(512, activation='relu'),
    Dense(2, activation='softmax') # 犬と猫を識別するため、２クラス分類のため2を指定 softmaxにする
])

#クラスを増やす参考URL
# https://aiacademy.jp/texts/show/?id=164





model_new.summary()

#model_new = Sequential([
#    Conv2D(16, 3, padding='same', activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH ,3)),
#    MaxPooling2D(),
#    Dropout(0.1),
#    Conv2D(32, 3, padding='same', activation='relu'),
#    MaxPooling2D(),
#    Conv2D(64, 3, padding='same', activation='relu'),
#    MaxPooling2D(),
#    Dropout(0.2),
#    Flatten(),
#    Dense(512, activation='relu'),
#    Dense(1, activation='sigmoid')
#])

### モデルのコンパイル

ネットワークにドロップアウトを導入した後、モデルをコンパイルし、レイヤーの概要を表示します。

optimizer参考
https://www.codetd.com/ja/article/9966554


kerasのConv2D（2次元畳み込み層）について調べてみた
https://qiita.com/kenichiro-yamato/items/60affeb7ca9f67c87a17

TensorFlowのaccuracyの値が少しも変動しません。
https://teratail.com/questions/106957

Kerasでval_acc、val_auc、val_lossが同じ値のまま更新されない
https://cocoinit23.com/keras-val-acc-val-auc-val-loss-same-value/

実務で使えるニューラルネットワークの最適化手法
https://acro-engineer.hatenablog.com/entry/2019/12/25/130000#Adam

オプティマイザの使用方法
https://tensorflow.classcat.com/2016/03/29/keras-optimizers/

http://tflearn.org/optimizers/

In [ ]:
#オプティマイズ調整


#授業より
# 乱数のシードを固定
#tf.random.set_seed(0)

# 定数（学習方法設計時に必要となるもの）
#LOSS = 'sparse_categorical_crossentropy'# 損失関数：クロスエントロピー
##LOSS = 'binary_crossentropy'
##METRICS = ['accuracy'] # 評価関数：正解率
##OPTIMIZER = tf.keras.optimizers.RMSprop #Adam #最適化で用いる手法
##LEARNING_RATE = 0.00001 # 学習率 # RMSprop(lr=0.00005, decay=1e-6)
##DECAY = 1e-6 #RMSprop

#BATCH_SIZE = 8 # バッチサイズ（損失関数を一度計算するのに使うデータの数）
#EPOCHS = 5000 # エポック数（全てのデータを使った学習を何回繰り返すか）
#VERBOSE = 0 # 学習の進捗状況の表示（0, 1または2。 0は表示なし, 2は詳細表示）

# 学習方法を定義する
##model_new.compile(loss=LOSS, # 損失関数
##              #optimizer=OPTIMIZER(learning_rate=LEARNING_RATE), # 最適化手法
##              optimizer=OPTIMIZER(learning_rate=LEARNING_RATE,decay=DECAY), # 最適化手法
##              metrics=METRICS) # 評価指標 ※リスト形式で設定
	
!pip install adabelief-tf==0.2.1

from adabelief_tf import AdaBeliefOptimizer

#adam = tf.keras.optimizers.Adam(lr=1e-5, beta1=0.99)

#OPTIMIZER = tf.keras.optimizers.Adam #RMSprop #最適化で用いる手法
#OPTIMIZER = tf.keras.optimizers.Adamax #RMSprop #最適化で用いる手法
LEARNING_RATE = 1e-6 # 学習率 # RMSprop(lr=0.00005, decay=1e-6)
DECAY = 1e-8 #RMSprop

#OPTIMIZER = AdaBeliefOptimizer(learning_rate=0.001, beta1=0.9, beta2=0.999, epsilon=1e-14, use_locking=False, name='AdaBelief', amsgrad=False)
#OPTIMIZER = AdaBeliefOptimizer(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-14, weight_decay=1e-4, rectify=True, amsgrad=False, sma_threshold=5.0, total_steps=0, warmup_proportion=0.1, min_lr=0.0, name='AdaBeliefOptimizer', print_change_log=True)
#eps=1e-16、weight_decay =1e-4、ouple betas=(0.9, 0.999) weight_decouple=True、rectify=True、
#ok bt8 OPTIMIZER = AdaBeliefOptimizer(learning_rate=1e-6, epsilon=1e-16, rectify=False)
#ok bt32 OPTIMIZER = AdaBeliefOptimizer(learning_rate=1e-6, epsilon=1e-16, rectify=False)

#以前はこれでうまくいっていた
OPTIMIZER = AdaBeliefOptimizer(learning_rate=1e-6, beta_1=0.9, beta_2=0.999, epsilon=1e-16, weight_decay=1e-5, rectify=True, amsgrad=False, sma_threshold=5.0, total_steps=0, warmup_proportion=0.1, min_lr=0.0, name='AdaBeliefOptimizer', print_change_log=True)

#OPTIMIZER = AdaBeliefOptimizer(learning_rate=1e-4,beta_1=0.9, beta_2=0.999, epsilon=1e-16,weight_decay=1e-5, rectify=True, amsgrad=False, sma_threshold=5.0,total_steps=10000,warmup_proportion=0.1,min_lr=1e-6, name='AdaBeliefOptimizer', print_change_log=True)


#OPTIMIZER = AdaBeliefOptimizer (lr=1e-3)

#model_new.compile(OPTIMIZER(learning_rate=LEARNING_RATE),
model_new.compile(OPTIMIZER,
                  #OPTIMIZER(learning_rate=LEARNING_RATE,decay=DECAY), # 最適化手法
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

#うまくいっているやつ
###model_new.compile(optimizer='adam',loss='binary_crossentropy', metrics=['accuracy'])




### モデルの学習

学習サンプルにデータ拡張を導入し、ネットワークにドロップアウトを追加した後、この新しいネットワークを学習します:

In [ ]:

##BATCH_SIZE = 128 # バッチサイズ（損失関数を一度計算するのに使うデータの数）
##EPOCHS = 20 # エポック数（全てのデータを使った学習を何回繰り返すか）
#VERBOSE = 0 # 学習の進捗状況の表示（0, 1または2。 0は表示なし, 2は詳細表示）


# モデルの学習開始(historyに学習過程のデータを保存)
##history = model_new.fit(train_data_gen,
##                    batch_size = BATCH_SIZE,
##                    epochs = EPOCHS,
                   # verbose = VERBOSE,
##                    steps_per_epoch=total_train // batch_size,
##                    validation_data=val_data_gen,
##                    validation_steps=total_val // batch_size
##                   )


history = model_new.fit_generator(
    train_data_gen,
    steps_per_epoch=total_train // batch_size,
    epochs=epochs,
    validation_data=val_data_gen,
    validation_steps=total_val // batch_size
)

#モデルの保存と復元方法
#https://www.tensorflow.org/tutorials/keras/save_and_load?hl=ja

# HDF5 ファイルにモデル全体を保存
# 拡張子 '.h5' はモデルが HDF5 で保存されているということを暗示する
#model.save('./my_model.h5')
model_new.save('./normal_r_l.h5')




### モデルの可視化

学習後に新しいモデルを可視化すると、過学習が前回よりも大幅に少ないことがわかります。より多くのエポックでモデルを学習すると、精度はさらに向上するはずです。

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
import PIL.Image
import numpy as np
import tensorflow_hub as hub
jpg = 0
if jpg == 0:
   image_path     = "./pred-Gamu1.jpg"
elif jpg == 1:
   image_path     = "./pred-Milk1.jpg" 


image = PIL.Image.open(image_path).convert("RGB").resize((128, 128))
image = np.array(image) / 255
image = np.expand_dims(image, 0)

#model_new = tf.keras.models.load_model("./my_model.h5", custom_objects={"KerasLayer": hub.KerasLayer})
model_pred = tf.keras.models.load_model("./normal_r_l.h5", compile=False , custom_objects={"KerasLayer": hub.KerasLayer})
#predictions = model_pred.predict(image)
print(predictions[0][0])
print(predictions)


#if predictions == 0:
#    print(">>> 犬")
#elif predictions == 1:
#    print(">>> 猫")

参考URL
TensorFlow2 + Keras による画像分類に挑戦4 ～学習済みモデルで予測させてみる～
https://qiita.com/code0327/items/1047adc050ab6d75ad5c